In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import calendar
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

class StockAnalyzer:
    def __init__(self, ticker: str, start_date: str = None, end_date: str = None):
        self.ticker = ticker
        self.start_date = start_date or (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')
        self.end_date = end_date or datetime.now().strftime('%Y-%m-%d')
        self.data = None
        self.metrics = {}
        
    def fetch_data(self):
        """Fetch stock data and calculate indicators"""
        try:
            self.data = yf.download(self.ticker, self.start_date, self.end_date)
            if self.data.empty:
                raise ValueError(f"No data found for {self.ticker}")
            
            self._calculate_indicators()
            return True
        except Exception as e:
            print(f"Error fetching data: {str(e)}")
            return False
    
    def _calculate_indicators(self):
        """Calculate technical indicators"""
        # Moving averages
        for window in [20, 50, 200]:
            self.data[f'MA{window}'] = self.data['Close'].rolling(window=window).mean()
        
        # RSI
        delta = self.data['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        self.data['RSI'] = 100 - (100 / (1 + rs))
        
        # MACD
        exp1 = self.data['Close'].ewm(span=12, adjust=False).mean()
        exp2 = self.data['Close'].ewm(span=26, adjust=False).mean()
        self.data['MACD'] = exp1 - exp2
        self.data['Signal_Line'] = self.data['MACD'].ewm(span=9, adjust=False).mean()
    
    def calculate_metrics(self):
        """Calculate performance metrics"""
        try:
            daily_returns = self.data['Close'].pct_change()
            
            self.metrics = {
                'Total_Return': (self.data['Close'][-1] / self.data['Close'][0] - 1) * 100,
                'Daily_Volatility': daily_returns.std(),
                'Annual_Volatility': daily_returns.std() * np.sqrt(252),
                'Sharpe_Ratio': (daily_returns.mean() / daily_returns.std()) * np.sqrt(252),
                'Max_Drawdown': ((self.data['Close'] / self.data['Close'].expanding().max()) - 1).min() * 100,
                'Current_Price': self.data['Close'][-1],
                'Starting_Price': self.data['Close'][0],
                'Highest_Price': self.data['High'].max(),
                'Lowest_Price': self.data['Low'].min(),
                'Average_Volume': self.data['Volume'].mean(),
                'RSI': self.data['RSI'][-1],
                'MACD_Signal': "Bullish" if self.data['MACD'][-1] > self.data['Signal_Line'][-1] else "Bearish"
            }
        except Exception as e:
            print(f"Error calculating metrics: {str(e)}")
    
    def create_plot(self):
        """Create analysis charts"""
        try:
            fig = make_subplots(
                rows=3, cols=1,
                shared_xaxes=True,
                vertical_spacing=0.05,
                subplot_titles=('Price & Moving Averages', 'Volume', 'Technical Indicators'),
                row_heights=[0.5, 0.2, 0.3]
            )
            
            # Candlestick chart
            fig.add_trace(go.Candlestick(
                x=self.data.index,
                open=self.data['Open'],
                high=self.data['High'],
                low=self.data['Low'],
                close=self.data['Close'],
                name='Price'
            ), row=1, col=1)
            
            # Moving averages
            colors = {'MA20': 'blue', 'MA50': 'orange', 'MA200': 'red'}
            for ma, color in colors.items():
                fig.add_trace(go.Scatter(
                    x=self.data.index,
                    y=self.data[ma],
                    name=ma,
                    line=dict(color=color)
                ), row=1, col=1)
            
            # Volume
            fig.add_trace(go.Bar(
                x=self.data.index,
                y=self.data['Volume'],
                name='Volume'
            ), row=2, col=1)
            
            # RSI
            fig.add_trace(go.Scatter(
                x=self.data.index,
                y=self.data['RSI'],
                name='RSI'
            ), row=3, col=1)
            
            # MACD
            fig.add_trace(go.Scatter(
                x=self.data.index,
                y=self.data['MACD'],
                name='MACD'
            ), row=3, col=1)
            
            fig.add_trace(go.Scatter(
                x=self.data.index,
                y=self.data['Signal_Line'],
                name='Signal Line'
            ), row=3, col=1)
            
            fig.update_layout(
                height=800,
                title_text=f"{self.ticker} Analysis Dashboard",
                showlegend=True,
                xaxis_rangeslider_visible=False
            )
            
            return fig
        except Exception as e:
            print(f"Error creating plot: {str(e)}")
            return None
    
    def generate_report(self):
        """Generate analysis report"""
        try:
            report = f"""
FINANCIAL ANALYSIS REPORT - {self.ticker}
===========================================
Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Analysis Period: {self.data.index[0].strftime('%Y-%m-%d')} to {self.data.index[-1].strftime('%Y-%m-%d')}

EXECUTIVE SUMMARY
----------------
Total Return: {self.metrics['Total_Return']:.2f}%
Current Trend: {"Upward" if self.data['Close'][-1] > self.data['MA20'][-1] else "Downward"}

PRICE PERFORMANCE
----------------
Current Price: ${self.metrics['Current_Price']:.2f}
Starting Price: ${self.metrics['Starting_Price']:.2f}
Highest Price: ${self.metrics['Highest_Price']:.2f}
Lowest Price: ${self.metrics['Lowest_Price']:.2f}
Average Daily Volume: {self.metrics['Average_Volume']:,.0f} shares

RISK METRICS
-----------
Daily Volatility: {self.metrics['Daily_Volatility']:.4f}
Annual Volatility: {self.metrics['Annual_Volatility']:.4f}
Sharpe Ratio: {self.metrics['Sharpe_Ratio']:.4f}
Maximum Drawdown: {self.metrics['Max_Drawdown']:.2f}%

TECHNICAL INDICATORS
------------------
RSI: {self.metrics['RSI']:.2f} ({'Overbought' if self.metrics['RSI'] > 70 else 'Oversold' if self.metrics['RSI'] < 30 else 'Neutral'})
MACD Signal: {self.metrics['MACD_Signal']}

TRADING IMPLICATIONS
------------------
{self._generate_trading_implications()}

RISK CONSIDERATIONS
------------------
Past performance does not guarantee future results. This analysis is based on historical data and should be used as one of many tools for investment decision-making.
"""
            return report
        except Exception as e:
            return f"Error generating report: {str(e)}"
    
    def _generate_trading_implications(self):
        """Generate trading implications based on technical indicators"""
        implications = []
        
        # RSI implications
        if self.metrics['RSI'] > 70:
            implications.append("- RSI indicates overbought conditions, suggesting caution for new long positions")
        elif self.metrics['RSI'] < 30:
            implications.append("- RSI indicates oversold conditions, potentially presenting buying opportunities")
        
        # MACD implications
        implications.append(
            f"- MACD shows {self.metrics['MACD_Signal'].lower()} momentum, " +
            ("supporting potential upward price movement" if self.metrics['MACD_Signal'] == "Bullish" 
             else "suggesting potential downward pressure")
        )
        
        # Trend implications
        if self.data['Close'][-1] > self.data['MA200'][-1]:
            implications.append("- Price is above 200-day MA, indicating a long-term uptrend")
        else:
            implications.append("- Price is below 200-day MA, indicating a long-term downtrend")
        
        return "\n".join(implications)

def analyze_stock(ticker: str, start_date: str = None, end_date: str = None):
    """Main function to perform stock analysis"""
    print(f"Analyzing {ticker}...")
    
    # Initialize analyzer
    analyzer = StockAnalyzer(ticker, start_date, end_date)
    
    # Fetch data
    if not analyzer.fetch_data():
        return
    
    # Calculate metrics
    analyzer.calculate_metrics()
    
    # Create and show plot
    fig = analyzer.create_plot()
    if fig:
        fig.show()
    
    # Generate and print report
    print("\n" + analyzer.generate_report())


    
    
    
    
    
    
    
    
    
    

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import calendar
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import warnings
warnings.filterwarnings('ignore')

class StockAnalysisWidget:
    def __init__(self):
        self._create_widgets()
        self._layout_widgets()
        
    def _create_widgets(self):
        """Create all interactive widgets"""
        # Ticker input
        self.ticker_input = widgets.Text(
            value='AAPL',
            placeholder='Enter stock ticker',
            description='Ticker:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px')
        )
        
        # Date range inputs
        self.start_date = widgets.DatePicker(
            description='Start Date:',
            value=datetime.now() - timedelta(days=365),
            style={'description_width': 'initial'}
        )
        
        self.end_date = widgets.DatePicker(
            description='End Date:',
            value=datetime.now(),
            style={'description_width': 'initial'}
        )
        
        # Analysis button
        self.analyze_button = widgets.Button(
            description='Analyze Stock',
            button_style='primary',
            icon='chart-line'
        )
        
        # Progress indicator
        self.progress = widgets.HTML(
            value="",
            layout=widgets.Layout(margin='10px 0px')
        )
        
        # Output area
        self.output = widgets.Output()
        
        # Add button click handler
        self.analyze_button.on_click(self._on_analyze_click)
        
    def _layout_widgets(self):
        """Arrange widgets in the layout"""
        # Input controls container
        self.controls = widgets.VBox([
            widgets.HBox([
                self.ticker_input,
                self.start_date,
                self.end_date,
                self.analyze_button
            ], layout=widgets.Layout(
                display='flex',
                flex_flow='row wrap',
                justify_content='flex-start',
                gap='10px',
                margin='10px 0px'
            )),
            self.progress
        ])
        
        # Main container
        self.container = widgets.VBox([
            self.controls,
            self.output
        ])
        
    def _on_analyze_click(self, b):
        """Handle analyze button click"""
        with self.output:
            clear_output(wait=True)
            
            # Update progress
            self.progress.value = "<b>Analysis in progress...</b>"
            
            try:
                # Create analyzer instance
                analyzer = StockAnalyzer(
                    self.ticker_input.value,
                    self.start_date.value.strftime('%Y-%m-%d'),
                    self.end_date.value.strftime('%Y-%m-%d')
                )
                
                # Fetch and analyze data
                if not analyzer.fetch_data():
                    self.progress.value = "<b style='color: red;'>Error fetching data</b>"
                    return
                
                # Calculate metrics
                analyzer.calculate_metrics()
                
                # Display results
                fig = analyzer.create_plot()
                if fig:
                    display(fig)
                
                # Display report
                report = analyzer.generate_report()
                display(HTML(f"""
                    <div style='
                        background-color: #f5f5f5;
                        padding: 20px;
                        border-radius: 5px;
                        margin: 20px 0;
                        font-family: monospace;
                        white-space: pre-wrap;
                    '>
                        {report}
                    </div>
                """))
                
                # Update progress
                self.progress.value = "<b style='color: green;'>Analysis complete!</b>"
                
            except Exception as e:
                self.progress.value = f"<b style='color: red;'>Error: {str(e)}</b>"
                print("\nPlease check:")
                print("1. Valid ticker symbol")
                print("2. Valid date range")
                print("3. Internet connection")
    
    def display(self):
        """Display the widget interface"""
        display(self.container)

# Function to initialize and display the widget
def create_stock_analyzer():
    app = StockAnalysisWidget()
    app.display()
    return app


  


  === Available Stock Symbols ===

Technology:
AAPL, MSFT, GOOGL, META, NVDA, AMD, INTC, CRM

Finance:
JPM, BAC, WFC, GS, MS, V, MA, AXP

Healthcare:
JNJ, PFE, MRK, UNH, ABBV, LLY, BMY, TMO

Consumer:
AMZN, WMT, PG, KO, PEP, MCD, NKE, SBUX

Industrial:
BA, CAT, GE, MMM, HON, UPS, FDX, LMT

Energy:
XOM, CVX, COP, SLB, EOG, MPC, PSX, VLO
    


In [8]:
# Example usage in Jupyter notebook
if __name__ == "__main__":
    app = create_stock_analyzer()